# George D. Crochiere - 0961739
## EE361 - Project 2
## 03/25/2024

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding
!pip install chainer
import cupy
# Note: if you cannot connect to GPU, then import cupy will not work

# from google.colab import drive
# drive.mount('/content/drive')
# Allow connection to Google Drive to save and access images

# Double Image

In [71]:
import cupy as cp
import time
from PIL import Image

# CUDA kernel code
double_kernel = cp.RawKernel(r'''
extern "C" __global__
void doubleImg(const unsigned char* old_im, unsigned char* new_im, int width, int height)
{
    int tid_x = blockDim.x * blockIdx.x + threadIdx.x;
    int tid_y = blockDim.y * blockIdx.y + threadIdx.y;

    for(int i = 0; i < 3; i++){
        for(int j = 0; j < 3; j++){
            int x = tid_x * 3 + i;
            int y = tid_y * 3 + j;

            if (x < (width) && y < (height)) {
                for (int c = 0; c < 3; c++) {
                    int tid = y * width * 3 + x * 3 + c;

                    unsigned char color = old_im[tid];

                    new_im[(y * 4) * width * 3 + (x * 2) * 3 + c] = color;
                    new_im[(y * 4) * width * 3 + (x * 2 + 1) * 3 + c] = color;
                    new_im[(y * 4 + 2) * width * 3 + (x * 2) * 3 + c] = color;
                    new_im[(y * 4 + 2) * width * 3 + (x * 2 + 1) * 3 + c] = color;
                }
            }
        }
    }
}

''', 'doubleImg')

def makeDouble(imageFile):
    # Load image using PIL
    oldImage = Image.open(imageFile).convert('RGB')
    width, height = oldImage.size

    # Convert image to cupy array
    oldIm = cp.asarray(oldImage, dtype=cp.ubyte)
    newIm = cp.empty_like(oldIm)

    # Reshape arrays for easier parallelization
    size = width * height * 3
    oldIm = cp.reshape(oldIm, (size,))
    newIm = cp.resize(newIm, (size * 4,))

    # Launch CUDA kernel
    block_dim = (32, 32)
    grid_dim = (int (cp.ceil(width/3/block_dim[0])), int (cp.ceil(height/3/block_dim[1])))

    t3 = time.perf_counter()
    double_kernel((grid_dim), (block_dim), (oldIm, newIm, width, height))
    t4 = time.perf_counter()
    print('time taken to run:',t4-t3)

    # Reshape newIm array back to image dimensions
    newIm = cp.reshape(newIm, (height * 2, width * 2, 3))

    # Convert cupy array to PIL Image
    newImage = Image.fromarray(cp.asnumpy(newIm))

    # Save the image
    newImage.save('./double.jpg')

if __name__ == '__main__':
    t1 = time.perf_counter()
    # input image
    makeDouble('./dog.jpg')
    t2 = time.perf_counter()
    print('time taken to run:',t2-t1)


time taken to run: 0.1799409000000196
time taken to run: 0.992564399999992


# Vertical Flip

In [27]:
import cupy
from PIL import Image

# CUDA Kernel - Vertical Flip
vertical_kernel_flip = cp.RawKernel(r'''
extern "C" __global__;
void verticalFlip(const unsigned char* old_im, unsigned char* new_im, width, height) {
    int tid_x = blockDim.x * blockIdx.x + threadIdx.x;
    int tid_y = blockDim.y * blockIdx.y + threadIdx.y;
    int size = width * height;
    
    for (int i = 0; i < 3; i++) {
        for (int j = 0; j < 3; j++) {
            int x = tid_x * 3 + i;
            int y = tid_y * 3 + j;
            
            if (x < width && y < height) {
                int tid = (y * width * 3) + (x * 3);
                
                unsigned char r = old_im[tid];
                unsigned char g = old_im[tid + 1];
                unsigned char b = old_im[tid + 2];
                
                int tid_new = ((width - y) * width * 3) + (x * 3);
                
                new_im[tid_new * 3] = r
                new_im[tid_new * 3 + 1] = g
                new_im[tid_new * 3 + 2] = b
            }
        }
    }
}
''', 'verticalFlip')

def makeVerticalFlip(imgFile):
    # Original Image
    oldImage = Image.open(imgFile).convert('RGB')
    width, height = oldImage.size
    
    oldImArr = cp.asarray(oldImage, (size, 3))
    newImArr = cp.empty_like(oldImArr)
    
    
    

# Edge Detection

In [ ]:
import cupy as cp
from PIL import Image

# CUDA kernel code
edge_kernel = cp.RawKernel(r'''
extern "C" __global__
void edgeDetection(const unsigned char* old_im, unsigned char* new_im, int size)
{
    
}

''', 'edgeDetection')
